# 评价方法分类
- 主观赋权法：综合指数法、模糊综合评判法、层次分析法、功效系数法
- 客观赋权法：主成分分析法、因子分析法、理想解法(TOPSIS)
# TOPSIS
## 定义
一种多指标评价方法，构造最优解和最劣解，通过计算到理想方案的贴近度来对方案进行排序，选出最佳方案。
## 步骤
1. 用向量规范化的方法求规范决策矩阵$B=(b_{ij})_{m\times n}$
$$b_{ij}=a_{ij}/\sqrt {\sum \limits_{i-1}^m a_{ij}^2 }$$
2. 构成加权规范化矩阵$C$
$$c_{ij}=w_{ij}\cdot b_{ij}$$
3. 确定正理想解$C^*$和负理想解$C^0$
4. 计算各方案到正理想解$s_i^*$与负理想解的距离$s_i^0$
5. 计算各方案的排队指标值$$f_i^*=s_i^0/(s_i^0+s_i^*)$$
6. 按$f_i^*$由大到小排列方案的优劣次序